# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__

Populating the interactive namespace from numpy and matplotlib
0.3.2
pysd/__init__.pyc


# Testing various subscript Options


In [6]:
from __future__ import division 
import numpy as np 
from pysd import functions
from scipy.integrate import odeint
import itertools

### option 1: unnamed array

In [23]:
class MinModel(object):
    ##########  boilerplate stuff from the existing pysd #########
    def __init__(self):
        self._stocknames = [name[:-5] for name in dir(self) if name[-5:] == '_init']
        self._stocknames.sort() #inplace
        self._dfuncs = [getattr(self, 'd%s_dt'%name) for name in self._stocknames]
        self.state = dict(zip(self._stocknames, [None]*len(self._stocknames)))
        self.reset_state()
        self.functions = functions.Functions(self)

    def reset_state(self):
        """Sets the model state to the state described in the model file. """
        self.t = self.initial_time() #set the initial time
        retry_flag = False
        for key in self.state.keys():
            try:
                self.state[key] = eval('self.'+key+'_init()') #set the initial state
            except TypeError:
                retry_flag = True
        if retry_flag:
            self.reset_state() #potential for infinite loop!

    ########### Stuff we have to modify to make subscripts work #########
    def d_dt(self, state_vector, t):
        """The primary purpose of this function is to interact with the integrator.
        It takes a state vector, sets the state of the system based on that vector,
        and returns a derivative of the state vector
        """        
        self.set_state(state_vector)
        self.t = t
        
        derivative_vector = []
        for func in self._dfuncs:
            derivative_vector += list(func())
            
        return derivative_vector

    def set_state(self, state_vector):
        i = 0
        for key in self._stocknames:
            if isinstance(self.state[key], np.ndarray):
                size = self.state[key].size
                elements = state_vector[i:i+size]
                shape = self.state[key].shape
                self.state[key] = np.array(elements).reshape(shape)
                i += size
            else:
                self.state[key] = state_vector[i]
                i += 1

        
    def get_state(self):
        #if we keep this, we should make it fully a list comprehension
        state_vector = []
        for item in [self.state[key] for key in self._stocknames]:
            if isinstance(item, np.ndarray):
                state_vector += list(item.flatten())
            else:
                state_vector += list(item)
        return state_vector
    
    
    ######### model specific components (that go in the model file)
    suba_list = ['suba1', 'suba2', 'suba3']
    subb_list = ['suba2', 'subb2']
    
    def stock(self, suba, subb):
        return self.state['stock'][self.suba_list.index(suba)][self.subb_list.index(subb)] 
    
    def stock_init(self):
        return np.array([[1,1],[1,1],[1,1]])
    
    def dstock_dt(self):
        return [self.flow(suba, subb) for suba, subb in itertools.product(self.suba_list, self.subb_list)]
    
    def constant(self, suba, subb):
        values = np.array([[1,2],[3,4],[5,6]])
        return values[self.suba_list.index(suba)][self.subb_list.index(subb)] 
    
    def flow(self, suba, subb):
        return self.constant(suba, subb) * self.stock(suba, subb)
    
    def initial_time(self):
        return 0

In [24]:
a = MinModel()

In [25]:
a.dstock_dt()

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

In [26]:
a.d_dt(a.get_state(), 0)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

In [27]:
odeint(a.d_dt, a.get_state(), range(10))

array([[  1.        ,   1.        ,   1.        ,   1.        ,
          1.        ,   1.        ],
       [  1.73205081,   2.23606799,   2.64575133,   3.00000004,
          3.31662484,   3.60555134],
       [  2.23606798,   3.00000003,   3.60555133,   4.1231057 ,
          4.58257579,   5.00000012],
       [  2.64575132,   3.60555132,   4.35889902,   5.0000001 ,
          5.56776449,   6.08276268],
       [  3.00000002,   4.12310568,   5.00000009,   5.74456277,
          6.40312439,   7.00000018],
       [  3.31662482,   4.58257576,   5.56776446,   6.40312437,
          7.14142859,   7.81024987],
       [  3.60555131,   5.00000008,   6.08276266,   7.00000016,
          7.81024987,   8.54400397],
       [  3.87298338,   5.38516489,   6.55743865,   7.5498346 ,
          8.42614997,   9.21954469],
       [  4.12310568,   5.74456275,   7.00000016,   8.06225795,
          9.00000023,   9.84885807],
       [  4.358899  ,   6.08276264,   7.41619865,   8.54400395,
          9.53939225,  10.4